In [67]:
import warnings
warnings.filterwarnings('ignore')

In [68]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [69]:
# Load data
file_path = Path('lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [70]:
# Fitting and encoding columns w/ LabelEncoder
le = LabelEncoder()

# Encoding homewowner column
le.fit(df["homeowner"])
df["homeowner"] = le.transform(df["homeowner"])

df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,1,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,1,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,2,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,1,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,0,53000,0.433962,5,1,23000,low_risk


In [71]:
# Create features
X = df.drop(columns="loan_status")

y=df["loan_status"]
X

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,1,52800,0.431818,5,1,22800
1,8400.0,6.692,1,43600,0.311927,3,0,13600
2,9000.0,6.963,2,46100,0.349241,3,0,16100
3,10700.0,7.664,1,52700,0.430740,5,1,22700
4,10800.0,7.698,0,53000,0.433962,5,1,23000
...,...,...,...,...,...,...,...,...
77531,19100.0,11.261,1,86600,0.653580,12,2,56600
77532,17700.0,10.662,0,80900,0.629172,11,2,50900
77533,17600.0,10.595,2,80300,0.626401,11,2,50300
77534,16300.0,10.068,0,75300,0.601594,10,2,45300


In [72]:
X.describe()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,0.606144,49221.949804,0.377318,3.826610,0.392308,19221.949804
std,2093.223153,0.889495,0.667811,8371.635077,0.081519,1.904426,0.582086,8371.635077
min,5000.000000,5.250000,0.000000,30000.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,0.000000,44800.000000,0.330357,3.000000,0.000000,14800.000000
50%,9500.000000,7.172000,1.000000,48100.000000,0.376299,4.000000,0.000000,18100.000000
75%,10400.000000,7.528000,1.000000,51400.000000,0.416342,4.000000,1.000000,21400.000000
max,23800.000000,13.235000,2.000000,105200.000000,0.714829,16.000000,3.000000,75200.000000


In [73]:
# Check balance of target values
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [74]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y,
                                                   random_state=1,
                                                   stratify=y)
X_train.shape

(58152, 8)

In [75]:
# Create scaler instance
scaler = StandardScaler()

# Fit scaler
X_scaler = scaler.fit(X_train)

# Scale data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [76]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train_scaled, y_train)

LogisticRegression(random_state=1)

In [77]:
# Calculate balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test_scaled)
bas=balanced_accuracy_score(y_test, y_pred)
print(bas)

0.9892813049736127


In [78]:
# Display confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"]
)
cm_df

,Predicted High Risk,Predicted Low Risk
Actual High Risk,615,10
Actual Low Risk,102,18657


In [79]:
# Print imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.98      0.99      0.92      0.99      0.98       625
   low_risk       1.00      0.99      0.98      1.00      0.99      0.98     18759

avg / total       0.99      0.99      0.98      0.99      0.99      0.98     19384



In [80]:
# Resample training data w/ RandomOverSampler
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)
X_resampled_ros, y_resampled_ros = ros.fit_resample(X_train_scaled, y_train)
Counter(y_resampled_ros)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [81]:
# Train Logistic Regression model w/ resampled data
from sklearn.linear_model import LogisticRegression

model_ros = LogisticRegression(solver='lbfgs', random_state=1)
model_ros.fit(X_resampled_ros, y_resampled_ros)

LogisticRegression(random_state=1)

In [82]:
# Calculate balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_pred_ros = model_ros.predict(X_test_scaled)

bas_ros = balanced_accuracy_score(y_test, y_pred_ros)
print(bas_ros)

0.9946414201183431


In [83]:
# Display confusion matrix
cm_ros = confusion_matrix(y_test, y_pred_ros)
cm_df_ros = pd.DataFrame(
    cm_ros, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"]
)
cm_df_ros

,Predicted High Risk,Predicted Low Risk
Actual High Risk,622,3
Actual Low Risk,111,18648


In [84]:
# Print imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred_ros))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



In [85]:
# Resample data using ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)
X_resampled_cc, y_resampled_cc = cc.fit_resample(X_train_scaled, y_train)

from collections import Counter

Counter(y_resampled_cc)

Counter({'high_risk': 1875, 'low_risk': 1875})

In [86]:
# train logistic Regression model using resampled data
from sklearn.linear_model import LogisticRegression
model_cc = LogisticRegression(solver='lbfgs', random_state=1)
model_cc.fit(X_resampled_cc, y_resampled_cc)

LogisticRegression(random_state=1)

In [87]:
# Calculate balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred_cc = model_cc.predict(X_test_scaled)
bas_cc = balanced_accuracy_score(y_test, y_pred_cc)
print(bas_cc)

0.9932813049736127


In [88]:
# Display confusion matrix
from sklearn.metrics import confusion_matrix
cm_cc = confusion_matrix(y_test, y_pred_cc)
cm_df_cc = pd.DataFrame(
    cm_cc, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"]
)
cm_df_cc

,Predicted High Risk,Predicted Low Risk
Actual High Risk,620,5
Actual Low Risk,102,18657


In [89]:
# Print imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred_cc))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.99      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18759

avg / total       1.00      0.99      0.99      0.99      0.99      0.99     19384



In [90]:
# Resample training data w/ SMOTE
from imblearn.over_sampling import SMOTE

X_resampled_smote, y_resampled_smote = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train_scaled, y_train
)
from collections import Counter

Counter(y_resampled_smote)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [91]:
# Train logistic regression model w/ resampled data
model_smote = LogisticRegression(solver='lbfgs', random_state=1)
model_smote.fit(X_resampled_smote, y_resampled_smote)

LogisticRegression(random_state=1)

In [92]:
# Calculate balanced accuracy score
y_pred_smote = model_smote.predict(X_test_scaled)
bas_smote=balanced_accuracy_score(y_test, y_pred_smote)
print(bas_smote)

0.9946414201183431


In [93]:
# Display confusion matrix
cm_smote = confusion_matrix(y_test, y_pred_smote)
cm_df_smote = pd.DataFrame(
    cm_smote, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"]
)
cm_df_smote

,Predicted High Risk,Predicted Low Risk
Actual High Risk,622,3
Actual Low Risk,111,18648


In [94]:
# Print imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_smote))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



In [95]:
# Resample training data w/ SMOTEENN
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_resampled_se, y_resampled_se = smote_enn.fit_resample(X_train_scaled, y_train)
Counter(y_resampled_se)

Counter({'high_risk': 55636, 'low_risk': 55913})

In [96]:
# Train logistic regression model w/ resampled data
from sklearn.linear_model import LogisticRegression
model_se = LogisticRegression(solver='lbfgs', random_state=1)
model_se.fit(X_resampled_se, y_resampled_se)

LogisticRegression(random_state=1)

In [97]:
# Calculate balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred_se = model_se.predict(X_test_scaled)
bas_se = balanced_accuracy_score(y_test, y_pred_se)
print(bas_se)

0.9946414201183431


In [98]:
# Display confusion matrix
cm_se = confusion_matrix(y_test, y_pred_se)
cm_df_se = pd.DataFrame(
    cm_se, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"]
)
cm_df_se

,Predicted High Risk,Predicted Low Risk
Actual High Risk,622,3
Actual Low Risk,111,18648


In [99]:
# Print imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred_se))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



In [100]:
print(f"Original dataset's balanced accuracy score = {bas}")
print(f"----------------------------------------------------")
print(f"Naive oversampling balanced accuracy score = {bas_ros}")
print(f"SMOTE balanced accuracy score = {bas_smote}")
print(f"SMOTEENN balanced accuracy score = {bas_se}")

Original dataset's balanced accuracy score = 0.9892813049736127
----------------------------------------------------
Naive oversampling balanced accuracy score = 0.9946414201183431
SMOTE balanced accuracy score = 0.9946414201183431
SMOTEENN balanced accuracy score = 0.9946414201183431


#### As indicated above, the Naive, SMOTE, and SMOTEENN models all had the same score, which was better than the original model.